# Modeling Sanity Check: Making sure everything is ok

In this notebook, we'll test our entire network pipeline because SURELY there are bugs.

In [ ]:
import dask.dataframe as dd
import pandas as pd 
import torch
import linecache 
import csv
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import sys
import torch
import pytorch_lightning as pl
sys.path.append('../src/')
sys.path.append('..')

Let's define our custom data class and make sure everything is being streamed in correctly

In [ ]:
from models.lib.data import GeneExpressionData
from models.lib.neural import GeneClassifier

In [ ]:
data = GeneExpressionData(
    filename='../data/interim/primary_bhaduri_T.csv',
    labelname='../data/processed/labels/primary_bhaduri_labels.csv',
    class_label='Type',
    skip=3,
)

data.shape

In [ ]:
data.labels

In [ ]:
model = GeneClassifier(
    input_dim = len(data.columns),
    output_dim = max(data.labels), # Since indexed from zero
#     weights=data.class_weights,
    optim_params={
        'optimizer': torch.optim.SGD,
        'lr': 0.001,
#         'momentum': 1e-4,
#         'weight_decay': 1e-4
    }
)

In [6]:
data.labels

array([16,  4,  9, 11,  6,  8,  7,  3, 17])

Now that we have our dataset, at least make sure a forward pass is computing correctly, and that our model can at least overfit on a small subset of the training set. Therefore, we'll subset our dataset and create the train and val loaders this way.

In [7]:
from torch.utils.data import Subset

tr_10k = Subset(data, range(1000))

In [8]:
def train_test(data):
    train_size = int(0.80 * len(data))
    test_size = len(data) - train_size

    train, test = torch.utils.data.random_split(data, [train_size, test_size])

    traindata = DataLoader(train, batch_size=4)
    valdata = DataLoader(test, batch_size=4)
    
    return traindata, valdata

train, test = train_test(tr_10k)

In [9]:
len(train.dataset), len(test.dataset)

(800, 200)

In [10]:
dir(train.dataset.dataset.dataset)

['__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cols',
 '_is_protocol',
 '_labeldf',
 'cast',
 'class_label',
 'class_weights',
 'columns',
 'features',
 'filename',
 'functions',
 'index_col',
 'indices',
 'labelname',
 'labels',
 'register_datapipe_as_function',
 'register_function',
 'sep',
 'shape',
 'skip']

In [11]:
train.dataset.dataset.dataset._labeldf['Type'].value_counts()

4     122958
16     29563
7      20609
8       4510
6       3863
17      2451
11      1888
9        363
3        271
Name: Type, dtype: int64

Even though we'll ultimately be using PyTorch Lightning for GPU training, let's try writing the training loop here so we can debug each step. To do this, we'll need to redefine the optimizer and loss

In [12]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

logger = WandbLogger()

run = Trainer(max_epochs=2000, logger=logger)
run.fit(model, train, test)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name       | Type                 | Params
----------------------------------------------------
0 | base_model | TabNetGeneClassifier | 1.3 M 
----------------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.241     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
